<a href="https://colab.research.google.com/github/CandidoJr209/Pytorch_examples/blob/main/Pytorch_DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MNIST digit classification with Dense Neural Network**

In this notebook we'll classify mnist digits using a DNN implemented with pytorch

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

First step, build the model

In [2]:
class NN(nn.Module):
  def __init__(self, input_size, num_classes):
    super(NN,self).__init__()
    self.fc1 = nn.Linear(input_size,50)
    self.fc2 = nn.Linear(50, num_classes)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x


Now select the device to use, that will be a cuda device if you have one and a cpu otherwise

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Tune the hiperparameters

In [4]:
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epoch = 1

Load the data

In [ ]:
train_dataset = datasets.MNIST(root='dataset/',train=True,transform=transforms.ToTensor(),download=True)
train_loader = DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_dataset = datasets.MNIST(root='dataset/',train=False,transform=transforms.ToTensor(),download=True)
test_loader = DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)

Now initialize the model with the parameters you chose

In [9]:
model = NN(input_size=input_size, num_classes=num_classes).to(device)

Define the loss and optimizer functions

In [10]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

Finally, train the model and display your results

In [11]:
for epoch in range(num_epoch):
  for batch_idx, (data, targets) in enumerate(train_loader):
    data = data.to(device=device)
    targets = targets.to(device=device)

    data = data.reshape(data.shape[0],-1)

    scores = model(data)
    loss = loss_fn(scores,targets)

    optimizer.zero_grad()
    loss.backward()

    optimizer.step()

def check_accuracy(loader, model):
  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    
    for x, y in loader:
      x = x.to(device=device)
      y = y.to(device=device)
      x = x.reshape(x.shape[0], -1)

      scores = model(x)
      _, predictions = scores.max(1)
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)

    print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

  model.train()


check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Got 55509 / 60000 with accuracy 92.52
Got 9293 / 10000 with accuracy 92.93
